In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import acnportal
import sklearn
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

from copy import deepcopy
import warnings
import pytz
import time
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from enum import Enum
from collections import namedtuple, defaultdict
import gzip
import random
import os
import json

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
# from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from utility_functions.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator
from acnportal.acnsim import analysis
# from modified_evse import *
from acnportal.acnsim.models.evse import get_evse_by_type

In [2]:
# This method won't take a evse_per_phase argument
def ev_fleet_level_2_network(transformer_cap=30):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'
    # evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(3)]
    BC_ids = ['BC-{0}'.format(i) for i in range(3)]
    CA_ids = ['CA-{0}'.format(i) for i in range(3)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

#### Default Simulation Parameter

In [3]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

#### Generate synthetic events for simulation

In [4]:
# Generate synthetic events based on JPL data provided by ACN-Sim.
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [5]:
# 25 EV sessions per weekday
events_25_ev = get_synth_events(25)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# Checking session details
session_copy = deepcopy(events_25_ev)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.arrival, \
          session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)

644 session_24 644 774 6.055058479912456 Plugin
654 session_21 654 768 15.384028380566479 Plugin
649 session_12 649 756 13.373157473859665 Plugin
662 session_18 662 784 12.018055601501445 Plugin
655 session_16 655 776 7.440790945476621 Plugin
655 session_23 655 774 14.67833190834886 Plugin
662 session_13 662 775 19.46010673624961 Plugin
669 session_4 669 773 13.128255680705868 Plugin
665 session_8 665 684 4.583215515438161 Plugin
669 session_9 669 792 6.582170038635213 Plugin
655 session_20 655 777 13.728554401109472 Plugin
656 session_5 656 803 12.271675837762457 Plugin
668 session_0 668 746 0.8736969369976232 Plugin
671 session_6 671 714 7.554759479182928 Plugin
755 session_14 755 791 10.420196328592658 Plugin
757 session_15 757 782 6.162466878486524 Plugin
717 session_1 717 779 13.200678512589743 Plugin
677 session_7 677 726 6.676963186383744 Plugin
669 session_17 669 781 11.599874773142352 Plugin
736 session_19 736 784 0.7413549136704182 Plugin
681 session_3 681 808 18.038775176580

#### Algorithms and new equation

In [8]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

#### Run Simulation with New Equation

In [9]:
sims = dict()
def run_experiment(alg_name, cap):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    start = timezone.localize(datetime(2024, 7, 31))
    end = timezone.localize(datetime(2024, 7, 31))

    cn = ev_fleet_level_2_network(transformer_cap=cap)

    alg = deepcopy(sch[alg_name])
    alg.max_rate_estimator = algorithms.SimpleRampdown()
    alg.estimate_max_rate = True
    experiment_events = deepcopy(events_25_ev)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = modified_simulator.Simulator(cn, alg, experiment_events, start, period=PERIOD, signals=signals, verbose=False)
    print("Running...")
    start_simulation = time.time()
    if alg_name == "MPC_Offline":
        alg.register_events(experiment_events)
        alg.solve()
    # if alg_name == "MPC":
    #     sim.high_priority_ev_sessions = ["session_2", "session_8", "session_19", "session_7"]
    # sim.high_priority_ev_sessions = ["session_5", "session_2", "session_19", "session_0", "session_6"]
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")

    return sim

In [10]:
warnings.simplefilter("ignore")

capacities = list(range(60, 90, 5))
alg_names = ["Unctrl", "RR", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.2695894241333008
('RR', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 1.366452693939209
('MPC', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 11.357816696166992
('Unctrl', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.2705378532409668
('RR', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 1.139373779296875
('MPC', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 11.22571611404419
('Unctrl', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.2851536273956299
('RR', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 1.336496353149414
('MPC', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 11.680231094360352
('Unctrl', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.2634305953979492
('RR', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 1.345489263534546
('MPC', 75)
60.240963

In [ ]:
# result_dir = "results/sims/new_simulation"
# if not os.path.exists(result_dir):
#     os.makedirs(result_dir)
    
# for config, sim in sims.items():
#     name = "results/sims/new_simulation/{0}-{1}.json.gz".format(*config)
#     if not os.path.exists(name):
#         data = sim.to_json()
#         with gzip.GzipFile(name, 'w') as fout:
#             fout.write(json.dumps(data).encode('utf-8'))

#### Result Analysis

In [11]:
def calc_metrics(config, sim):
    metrics = {
        "Network Type": "three_phase",
        "Algorithm": config[0],
        "Capacity (kW)": config[1],
        "Energy Delivered (%)": analysis.proportion_of_energy_delivered(sim) * 100,
        "Demand met": analysis.proportion_of_demands_met(sim) * 100,
        "Max Utilization (%)": np.max(analysis.aggregate_power(sim)) / config[1] * 100,
        "energy_cost": analysis.energy_cost(sim),
        "total_energy_delivered": analysis.total_energy_delivered(sim),
        "Peak (kW)": np.max(analysis.aggregate_power(sim))
    }
    metrics["Current Unbalance"] = np.nanmean(analysis.current_unbalance(sim, ['Secondary {0}'.format(p) for p in 'ABC'], 'NEMA'))

    return metrics

In [12]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [13]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Peak (kW),Current Unbalance
0,three_phase,Unctrl,60,99.995,100.0,165.000,26.631,253.245,99.000,0.387
1,three_phase,RR,60,93.149,76.0,103.058,33.648,235.907,61.835,0.391
2,three_phase,MPC,60,73.627,48.0,98.567,21.929,186.465,59.140,0.409
3,three_phase,Unctrl,65,99.995,100.0,152.308,26.631,253.245,99.000,0.375
4,three_phase,RR,65,75.541,56.0,102.800,24.081,191.313,66.820,0.358
5,three_phase,MPC,65,70.718,48.0,95.777,19.808,179.098,62.255,0.401
6,three_phase,Unctrl,70,99.995,100.0,141.429,26.791,253.245,99.000,0.372
7,three_phase,RR,70,93.861,84.0,100.793,31.408,237.710,70.555,0.409
8,three_phase,MPC,70,92.865,76.0,96.050,28.173,235.187,67.235,0.404
9,three_phase,Unctrl,75,99.995,100.0,132.000,26.631,253.245,99.000,0.353


#### Simulation with different algorithms

In [14]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

In [15]:
sims = dict()

In [16]:
warnings.simplefilter("ignore")

capacities = list(range(60, 90, 5))
alg_names = ["Unctrl", "LLF", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.3327198028564453
('LLF', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.7671453952789307
('MPC', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 11.566824913024902
('Unctrl', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.29994869232177734
('LLF', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.8007199764251709
('MPC', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 11.046676874160767
('Unctrl', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.2833442687988281
('LLF', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.6923189163208008
('MPC', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 11.353801488876343
('Unctrl', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.2701833248138428
('LLF', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.7289767265319824
('MPC', 75)

In [17]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [18]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Peak (kW),Current Unbalance
0,three_phase,Unctrl,60,99.995,100.0,165.000,26.631,253.245,99.000,0.380
1,three_phase,LLF,60,93.492,76.0,79.208,34.488,236.774,47.525,0.346
2,three_phase,MPC,60,92.903,80.0,97.875,30.535,235.283,58.725,0.410
3,three_phase,Unctrl,65,99.995,100.0,152.308,26.791,253.245,99.000,0.378
4,three_phase,LLF,65,94.153,92.0,90.992,32.258,238.449,59.145,0.392
5,three_phase,MPC,65,81.958,72.0,99.285,25.116,207.563,64.535,0.429
6,three_phase,Unctrl,70,99.995,100.0,141.429,26.631,253.245,99.000,0.375
7,three_phase,LLF,70,94.200,92.0,93.093,31.852,238.568,65.165,0.379
8,three_phase,MPC,70,82.001,68.0,96.643,24.855,207.673,67.650,0.426
9,three_phase,Unctrl,75,99.995,100.0,132.000,26.631,253.245,99.000,0.363
